# This notebook preprocesses variant data. Before using this notebook, you must run the GATK ApplyRecalibration tool (ie VQSR) as follows:


# Then you must use GATK's VariantsToTable tool from the command line as follows:

### The code below performs additional preprocessing on the table generated by VariantsToTable.

In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

## Ignore columns that are nonnumeric for now. Also drop AN and MQ0 which have no variance

In [2]:
ignore_cols = ['sample1.PL', 'sample1.AD', 'sample1.GT', 'REF', 'ALT', 'ID', 'CHROM', 'POS', 'AN', 'MQ0']

## Remove NANs from the VQSR-filtered SNP table. This table wil be useful in case we wish to use features computed by GATK (e.g. ReadPosRankSum) for our own algorithm.

In [4]:
filename = 'NA12878.LowSeq.illumina.bwa.sorted.dedup.11.sam.wFlag.qual.recalibrated.filtered'
in_file = '../../vqsr_output/chrom11/' + filename + '.table' #'../data/vqsr_output_all_chrom/' + filename + '.table'

df_orig = pd.read_csv(in_file, sep='\t')
df = df_orig.drop(ignore_cols, axis=1)
df_filter = df['FILTER']

# The POSITIVE_TRAIN_SITE and DB (dbSNP membership) columns are either true or NA. Convert True to 1 and NA to 0. 
df['POSITIVE_TRAIN_SITE'] = 1*df['POSITIVE_TRAIN_SITE'].fillna(0)
num_pos_train_sites = df['POSITIVE_TRAIN_SITE'].sum()
df['DB'] = 1*df['DB'].fillna(0)
num_dbSNP = df['DB'].sum()

print df.columns
print df.shape
print 'Current number of features (before removing NaNs): ', df.shape[1]
print 'Total number of variants: ', df.shape[0]
print 'VQSR used ', num_pos_train_sites, ' variants for training. As a pct: ', 100.0*num_pos_train_sites/df.shape[0]
print 'Pct of variants in dbSNP ', 100.0*num_dbSNP/df.shape[0]

Index([u'QUAL', u'FILTER', u'AC', u'AF', u'BaseQRankSum', u'ClippingRankSum',
       u'DB', u'DP', u'FS', u'MLEAC', u'MLEAF', u'MQ', u'MQRankSum',
       u'POSITIVE_TRAIN_SITE', u'QD', u'ReadPosRankSum', u'SOR', u'VQSLOD',
       u'culprit', u'FORMAT', u'sample1', u'sample1.DP', u'sample1.GQ'],
      dtype='object')
(187242, 23)
Current number of features (before removing NaNs):  23
Total number of variants:  187242
VQSR used  153698  variants for training. As a pct:  82.0852159238
Pct of variants in dbSNP  98.4538725286


In [5]:
df.describe()

/Users/nicochaves/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,QUAL,AC,AF,BaseQRankSum,ClippingRankSum,DP,FS,MLEAC,MLEAF,MQ,MQRankSum,QD,ReadPosRankSum,SOR,VQSLOD,FORMAT,sample1,sample1.DP,sample1.GQ
count,187242.000000,187242.000000,187242.000000,109756.000000,109756.000000,187242.000000,187242.000000,187242.000000,187242.000000,187242.000000,109756.000000,187242.000000,109697.000000,187242.000000,183713.000000,0.0,0.0,187242.000000,187242.000000
mean,317.913841,1.415767,0.707883,-0.022772,-0.021392,14.578535,1.620895,1.415671,0.707835,58.494981,-0.146604,20.858594,0.117065,1.143479,14.653301,NaN,NaN,14.359599,71.151446
std,288.665872,0.492855,0.246428,1.298535,0.899010,10.387486,3.484486,0.492839,0.246419,4.858719,1.035364,10.283196,0.979537,0.705455,7.205113,NaN,NaN,7.832364,29.629664
min,10.200000,1.000000,0.500000,-6.966000,-3.599000,1.000000,0.000000,1.000000,0.500000,20.000000,-7.739000,0.020000,-6.937000,0.006000,-380.100000,NaN,NaN,1.000000,0.000000
25%,156.770000,1.000000,0.500000,NaN,NaN,11.000000,0.000000,1.000000,0.500000,60.000000,NaN,12.060000,NaN,0.693000,NaN,NaN,NaN,11.000000,42.000000
50%,265.780000,1.000000,0.500000,NaN,NaN,14.000000,0.000000,1.000000,0.500000,60.000000,NaN,18.870000,NaN,0.991000,NaN,NaN,NaN,14.000000,84.000000
75%,447.770000,2.000000,1.000000,NaN,NaN,17.000000,2.276000,2.000000,1.000000,60.000000,NaN,31.750000,NaN,1.445000,NaN,NaN,NaN,17.000000,99.000000
max,18645.770000,2.000000,1.000000,7.142000,3.383000,759.000000,228.098000,2.000000,1.000000,70.000000,4.787000,41.470000,7.272000,8.887000,23.590000,NaN,NaN,424.000000,99.000000


In [6]:
df.dropna(axis='columns', how='any', inplace=True)
print df.columns
df.describe()

Index([u'QUAL', u'FILTER', u'AC', u'AF', u'DB', u'DP', u'FS', u'MLEAC',
       u'MLEAF', u'MQ', u'POSITIVE_TRAIN_SITE', u'QD', u'SOR', u'sample1.DP',
       u'sample1.GQ'],
      dtype='object')


,QUAL,AC,AF,DP,FS,MLEAC,MLEAF,MQ,QD,SOR,sample1.DP,sample1.GQ
count,187242.000000,187242.000000,187242.000000,187242.000000,187242.000000,187242.000000,187242.000000,187242.000000,187242.000000,187242.000000,187242.000000,187242.000000
mean,317.913841,1.415767,0.707883,14.578535,1.620895,1.415671,0.707835,58.494981,20.858594,1.143479,14.359599,71.151446
std,288.665872,0.492855,0.246428,10.387486,3.484486,0.492839,0.246419,4.858719,10.283196,0.705455,7.832364,29.629664
min,10.200000,1.000000,0.500000,1.000000,0.000000,1.000000,0.500000,20.000000,0.020000,0.006000,1.000000,0.000000
25%,156.770000,1.000000,0.500000,11.000000,0.000000,1.000000,0.500000,60.000000,12.060000,0.693000,11.000000,42.000000
50%,265.780000,1.000000,0.500000,14.000000,0.000000,1.000000,0.500000,60.000000,18.870000,0.991000,14.000000,84.000000
75%,447.770000,2.000000,1.000000,17.000000,2.276000,2.000000,1.000000,60.000000,31.750000,1.445000,17.000000,99.000000
max,18645.770000,2.000000,1.000000,759.000000,228.098000,2.000000,1.000000,70.000000,41.470000,8.887000,424.000000,99.000000


## Scale/normalize the VQSR-filtered SNP table (except categorical features)

In [7]:
def scale_data_frame(df):
    return (df - df.mean()) / (df.max() - df.min())

In [8]:
# Categorical features
cats = ['POSITIVE_TRAIN_SITE', 'DB', 'FILTER']

df_scaled = scale_data_frame(df.drop(cats, axis=1)) 
df_scaled[cats] = df[cats]

df_scaled.describe()

,QUAL,AC,AF,DP,FS,MLEAC,MLEAF,MQ,QD,SOR,sample1.DP,sample1.GQ
count,1.872420e+05,1.872420e+05,1.872420e+05,1.872420e+05,1.872420e+05,1.872420e+05,1.872420e+05,1.872420e+05,1.872420e+05,1.872420e+05,1.872420e+05,1.872420e+05
mean,-6.263256e-14,2.419946e-14,2.419946e-14,1.489728e-16,-3.212201e-17,1.344049e-15,1.344049e-15,-8.068610e-16,-7.060106e-15,3.923935e-14,-1.620469e-16,6.513593e-15
std,1.549005e-02,4.928550e-01,4.928550e-01,1.370381e-02,1.527627e-02,4.928386e-01,4.928386e-01,9.717438e-02,2.480868e-01,7.943421e-02,1.851623e-02,2.992895e-01
min,-1.651218e-02,-4.157668e-01,-4.157668e-01,-1.791363e-02,-7.106135e-03,-4.156706e-01,-4.156706e-01,-7.698996e-01,-5.027405e-01,-1.280801e-01,-3.158298e-02,-7.187015e-01
25%,-8.647111e-03,-4.157668e-01,-4.157668e-01,-4.721022e-03,-7.106135e-03,-4.156706e-01,-4.156706e-01,3.010037e-02,-2.122701e-01,-5.072395e-02,-7.942314e-03,-2.944590e-01
50%,-2.797545e-03,-4.157668e-01,-4.157668e-01,-7.632384e-04,-7.106135e-03,-4.156706e-01,-4.156706e-01,3.010037e-02,-4.797573e-02,-1.716917e-02,-8.501154e-04,1.297834e-01
75%,6.968188e-03,5.842332e-01,5.842332e-01,3.194545e-03,2.872033e-03,5.843294e-01,5.843294e-01,3.010037e-02,2.627601e-01,3.395120e-02,6.242083e-03,2.812985e-01
max,9.834878e-01,5.842332e-01,5.842332e-01,9.820864e-01,9.928939e-01,5.843294e-01,5.843294e-01,2.301004e-01,4.972595e-01,8.719199e-01,9.684170e-01,2.812985e-01


In [9]:
df_scaled.max()

QUAL                                      0.983488
AC                                        0.584233
AF                                        0.584233
DP                                        0.982086
FS                                        0.992894
MLEAC                                     0.584329
MLEAF                                     0.584329
MQ                                          0.2301
QD                                        0.497259
SOR                                        0.87192
sample1.DP                                0.968417
sample1.GQ                                0.281299
POSITIVE_TRAIN_SITE                              1
DB                                               1
FILTER                 VQSRTrancheSNP99.90to100.00
dtype: object

In [10]:
out_file = '../../data/processed/chrom11/' + filename + '.noNAN.scaled.table'
df_scaled.to_csv(out_file, sep='\t')

## Append Ground Truth data to the VQSR-filtered table

In [11]:
# Build dictionary of ground truth variants
gt_file = '../../data/processed/chrom11/ground_truth_chrom_11.txt'
gt_df = pd.read_csv(gt_file, sep='\t')
gnd_truth_dict = {}

for index, row in gt_df.iterrows(): 
    pos = row['POS']
    ref = row['REF']
    alt = row['ALT']
    gnd_truth_dict[pos] = (ref, alt)

In [12]:
gt_df.shape

(213022, 11)

### Note that there are more ground truth variant calls than variant calls in the non-ground-truth dataset. This reflects the fact that the sequencing pipeline doesn't discover all of the true variants. 

In [13]:
# Initialize the ground truth column to all 0's
df_scaled['GROUND_TRUTH'] = 0

# Lookup each variant in the VCF data and label it as a true variant or not
for index, row in df_orig.iterrows():   
    pos = row['POS']
    ref = row['REF']
    alt = row['ALT']
    
    if pos in gnd_truth_dict and gnd_truth_dict[pos] == (ref, alt):
        df_scaled.set_value(index, 'GROUND_TRUTH', 1)


In [14]:
filename = filename + '.noNAN.scaled.withGndTruthLabels'
out_file_with_labels = '../../data/processed/chrom11/' + filename + '.table'
df_scaled.to_csv(out_file_with_labels, sep='\t')

In [15]:
num_true_variants = sum(df_scaled['GROUND_TRUTH'])
true_pos_unfiltered = 1.0*num_true_variants/df_scaled.shape[0]
print 'Sensitivity (True Pos Rate) Before Any Filtering: ', true_pos_unfiltered

Sensitivity (True Pos Rate) Before Any Filtering:  0.934581984811


### Important! 93.5% of the variants called in the chromosome 11 VCF file are true variants according to the ground truth. So any nontrivial algorithm should have a specificity less than 6%. If we assume that our VCF file contains all of the true pos variants, then our algorithm should attain a sensitivity > 93.5%.

## Split data into training (70%) and test (30%) sets 

In [16]:
num_samples = df_scaled.shape[0]
idx_train_test = train_test_split(range(num_samples), range(num_samples), test_size = 0.3, random_state=0)

In [17]:
idx_train = idx_train_test[0]
idx_test = idx_train_test[1]
assert bool(set(idx_train) & set(idx_test)) == False

df_train = df_scaled.loc[idx_train,:]
df_test = df_scaled.loc[idx_test,:]

num_train = df_train.shape[0]
num_test = df_test.shape[0]

assert num_train + num_test == num_samples

print 'Training set size: ' + str(num_train)
print 'Test set size: ' + str(num_test)

Training set size: 131069
Test set size: 56173


## Save the training and test sets

In [18]:
df_train.to_csv('../../data/processed/chrom11/' + filename + '.train.table', sep='\t')
df_test.to_csv('../../data/processed/chrom11/' + filename + '.test.table', sep='\t')